In [10]:
# Import libraries
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
sys.path.append('..')

# Import custom FastF1 loader
from src.fastf1_loader import (
    get_session, get_race_results, get_lap_data,
    get_telemetry, get_weather_data, get_schedule,
    compare_drivers_lap_times
)

# Enable FastF1 cache
fastf1.Cache.enable_cache('../cache')

plt.style.use('seaborn-v0_8-darkgrid')
print('FastF1 loaded successfully!')
print(f'FastF1 version: {fastf1.__version__}')

FastF1 loaded successfully!
FastF1 version: 3.7.0


In [11]:
# Environment Setup and Library Imports
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
from pathlib import Path

warnings.filterwarnings('ignore')

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

# Enable FastF1 cache for faster data loading
cache_dir = Path.cwd().parent / 'cache'
cache_dir.mkdir(exist_ok=True)
fastf1.Cache.enable_cache(str(cache_dir))

print('FastF1 Telemetry Analysis Environment Ready')
print(f'FastF1 Version: {fastf1.__version__}')
print(f'Cache Directory: {cache_dir}')

FastF1 Telemetry Analysis Environment Ready
FastF1 Version: 3.7.0
Cache Directory: c:\Users\test\OneDrive\Desktop\PYTHON\f1 visualization\cache


## 1. Load 2025 Season Schedule

Get the official F1 2025 calendar from FastF1 API.

In [12]:
# Get 2025 F1 Schedule
def safe_get_schedule(year: int) -> pd.DataFrame:
    """Safely load F1 schedule with error handling."""
    try:
        schedule = fastf1.get_event_schedule(year)
        return schedule
    except Exception as e:
        print(f'Error loading schedule: {e}')
        return pd.DataFrame()

schedule = safe_get_schedule(2025)

if not schedule.empty:
    print('2025 F1 Calendar:')
    display(schedule[['RoundNumber', 'EventName', 'Country', 'Location', 'EventDate']].head(24))
else:
    print('Could not load schedule')

2025 F1 Calendar:


,RoundNumber,EventName,Country,Location,EventDate
0,0,Pre-Season Testing,Bahrain,Sakhir,2025-02-28
1,1,Australian Grand Prix,Australia,Melbourne,2025-03-16
2,2,Chinese Grand Prix,China,Shanghai,2025-03-23
3,3,Japanese Grand Prix,Japan,Suzuka,2025-04-06
4,4,Bahrain Grand Prix,Bahrain,Sakhir,2025-04-13
5,5,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,2025-04-20
6,6,Miami Grand Prix,United States,Miami Gardens,2025-05-04
7,7,Emilia Romagna Grand Prix,Italy,Imola,2025-05-18
8,8,Monaco Grand Prix,Monaco,Monaco,2025-05-25
9,9,Spanish Grand Prix,Spain,Barcelona,2025-06-01


## 2. Load Race Session Data

Load a race session with proper error handling and status messages.

In [13]:
# Safe session loading function
def safe_load_session(year: int, race: str, session_type: str = 'R'):
    """
    Safely load F1 session with error handling.
    
    Parameters:
    - year: Season year (e.g., 2025)
    - race: Race name or round number
    - session_type: 'R' (Race), 'Q' (Qualifying), 'FP1', 'FP2', 'FP3', 'S' (Sprint)
    
    Returns:
    - FastF1 Session object or None
    """
    try:
        print(f'Loading {year} {race} - {session_type}...')
        session = fastf1.get_session(year, race, session_type)
        session.load()
        print(f'Session loaded: {session.event["EventName"]} - {session.name}')
        print(f'Date: {session.date}')
        return session
    except Exception as e:
        print(f'Error loading session: {e}')
        print('Tip: Check if the race has occurred and data is available.')
        return None

# Load Australia 2025 Race (first race of season)
session = safe_load_session(2025, 'Australia', 'R')

Loading 2025 Australia - R...


core           INFO 	Loading data for Australian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core  

Session loaded: Australian Grand Prix - Race
Date: 2025-03-16 04:00:00


In [14]:
# Get race results with error handling
def get_race_results(session) -> pd.DataFrame:
    """Extract race results from session."""
    if session is None:
        print('No session available')
        return pd.DataFrame()
    try:
        results = session.results.copy()
        cols = ['Position', 'Abbreviation', 'FullName', 'TeamName', 'Time', 'Status', 'Points']
        available_cols = [c for c in cols if c in results.columns]
        return results[available_cols]
    except Exception as e:
        print(f'Error getting results: {e}')
        return pd.DataFrame()

results = get_race_results(session)
if not results.empty:
    print('Race Results:')
    display(results)

Race Results:


,Position,Abbreviation,FullName,TeamName,Time,Status,Points
4,1.0,NOR,Lando Norris,McLaren,0 days 01:42:06.304000,Finished,25.0
1,2.0,VER,Max Verstappen,Red Bull Racing,0 days 00:00:00.895000,Finished,18.0
63,3.0,RUS,George Russell,Mercedes,0 days 00:00:08.481000,Finished,15.0
12,4.0,ANT,Andrea Kimi Antonelli,Mercedes,0 days 00:00:10.135000,Finished,12.0
23,5.0,ALB,Alexander Albon,Williams,0 days 00:00:12.773000,Finished,10.0
18,6.0,STR,Lance Stroll,Aston Martin,0 days 00:00:17.413000,Finished,8.0
27,7.0,HUL,Nico Hulkenberg,Kick Sauber,0 days 00:00:18.423000,Finished,6.0
16,8.0,LEC,Charles Leclerc,Ferrari,0 days 00:00:19.826000,Finished,4.0
81,9.0,PIA,Oscar Piastri,McLaren,0 days 00:00:20.448000,Finished,2.0
44,10.0,HAM,Lewis Hamilton,Ferrari,0 days 00:00:22.473000,Finished,1.0


## 3. Lap Time Analysis

Analyze lap times for all drivers throughout the race.

In [15]:
# Get lap data with error handling
def get_lap_data(session) -> pd.DataFrame:
    """Extract lap data from session."""
    if session is None:
        return pd.DataFrame()
    try:
        laps = session.laps.copy()
        print(f'Total laps recorded: {len(laps)}')
        return laps
    except Exception as e:
        print(f'Error getting lap data: {e}')
        return pd.DataFrame()

laps = get_lap_data(session)
if not laps.empty:
    display(laps.head())

Total laps recorded: 927


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:13:00.002000,VER,1,0 days 00:01:59.392000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:20.705000,...,True,Red Bull Racing,0 days 01:11:00.355000,2025-03-16 04:18:22.974,124,2.0,False,,False,False
1,0 days 01:15:49.358000,VER,1,NaT,2.0,1.0,NaT,0 days 01:15:38.205000,0 days 00:00:58.141000,0 days 00:00:37.976000,...,True,Red Bull Racing,0 days 01:13:00.002000,2025-03-16 04:20:22.621,4,2.0,False,,False,False
2,0 days 01:18:31.526000,VER,1,NaT,3.0,2.0,0 days 01:15:51.658000,0 days 01:18:20.223000,0 days 00:00:56.230000,0 days 00:00:33.683000,...,False,Red Bull Racing,0 days 01:15:49.358000,2025-03-16 04:23:11.977,4,2.0,False,,False,False
3,0 days 01:21:07.226000,VER,1,NaT,4.0,3.0,0 days 01:18:34.029000,0 days 01:20:56.543000,0 days 00:00:54.351000,0 days 00:00:32.712000,...,False,Red Bull Racing,0 days 01:18:31.526000,2025-03-16 04:25:54.145,4,2.0,False,,False,False
4,0 days 01:23:30.835000,VER,1,0 days 00:02:23.609000,5.0,4.0,0 days 01:21:09.534000,NaT,0 days 00:00:53.513000,0 days 00:00:32.627000,...,False,Red Bull Racing,0 days 01:21:07.226000,2025-03-16 04:28:29.845,4,2.0,False,,False,False


In [16]:
# Convert lap times and filter valid laps
if not laps.empty and 'LapTime' in laps.columns:
    # Convert to seconds
    laps['LapTime_sec'] = laps['LapTime'].dt.total_seconds()
    
    # Filter valid laps (remove outliers: pit laps, SC, etc.)
    lap_median = laps['LapTime_sec'].median()
    valid_laps = laps[
        (laps['LapTime_sec'] > lap_median * 0.9) & 
        (laps['LapTime_sec'] < lap_median * 1.2)
    ].copy()
    
    print(f'Valid laps: {len(valid_laps)} / {len(laps)}')
    
    # Average lap time per driver
    avg_lap_times = valid_laps.groupby('Driver')['LapTime_sec'].mean().sort_values()
    
    fig = px.bar(
        avg_lap_times.reset_index(name='Avg Lap Time (sec)'),
        x='Avg Lap Time (sec)', y='Driver', orientation='h',
        color='Avg Lap Time (sec)', color_continuous_scale='RdYlGn_r',
        title='Average Lap Time per Driver'
    )
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, template='plotly_dark')
    fig.show()
else:
    print('No lap data available')

Valid laps: 616 / 927


In [17]:
# Lap time evolution for top 3 drivers
if not results.empty and not valid_laps.empty:
    top_3_drivers = results.head(3)['Abbreviation'].tolist()
    top_3_laps = valid_laps[valid_laps['Driver'].isin(top_3_drivers)]
    
    if not top_3_laps.empty:
        fig = px.line(
            top_3_laps, x='LapNumber', y='LapTime_sec', color='Driver',
            title='Lap Time Evolution - Top 3 Finishers',
            markers=True
        )
        fig.update_layout(
            template='plotly_dark',
            xaxis_title='Lap',
            yaxis_title='Lap Time (sec)'
        )
        fig.show()
else:
    print('No data available for lap evolution chart')

## 4. Fastest Laps Comparison

Compare fastest laps and gaps between drivers.

In [18]:
# Get fastest lap for each driver
if not laps.empty and 'LapTime_sec' in laps.columns:
    # Find index of fastest lap per driver
    idx = laps.groupby('Driver')['LapTime_sec'].idxmin()
    fastest_laps = laps.loc[idx].copy()
    fastest_laps = fastest_laps.sort_values('LapTime_sec')
    
    # Calculate gap to overall fastest
    fastest_overall = fastest_laps['LapTime_sec'].min()
    fastest_laps['Gap_to_Fastest'] = fastest_laps['LapTime_sec'] - fastest_overall
    
    print('Fastest Lap per Driver:')
    display(fastest_laps[['Driver', 'Team', 'LapNumber', 'LapTime_sec', 'Compound', 'Gap_to_Fastest']].head(10))
else:
    print('No lap data available')

KeyError: '[nan] not in index'

In [ ]:
# Gap to fastest lap visualization
if 'fastest_laps' in dir() and not fastest_laps.empty:
    fig = px.bar(
        fastest_laps, x='Gap_to_Fastest', y='Driver', orientation='h',
        color='Team', title='Gap to Fastest Lap'
    )
    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        template='plotly_dark',
        xaxis_title='Gap (seconds)'
    )
    fig.show()

## 5. Tyre Strategy Analysis

Analyze tyre compound usage and stint lengths.

In [ ]:
# Compound usage per driver
if not laps.empty and 'Compound' in laps.columns:
    compound_usage = laps.groupby(['Driver', 'Compound']).size().unstack(fill_value=0)
    
    fig = px.imshow(
        compound_usage,
        title='Tyre Compound Usage (Laps per Compound)',
        labels={'color': 'Laps'},
        color_continuous_scale='YlOrRd'
    )
    fig.update_layout(template='plotly_dark')
    fig.show()
else:
    print('No compound data available')

In [ ]:
# Stint length analysis by compound
if not laps.empty and 'Stint' in laps.columns:
    stint_data = laps.groupby(['Driver', 'Stint'])['Compound'].first().reset_index()
    stint_lengths = laps.groupby(['Driver', 'Stint']).size().reset_index(name='StintLength')
    stint_data = stint_data.merge(stint_lengths, on=['Driver', 'Stint'])
    
    fig = px.box(
        stint_data, x='Compound', y='StintLength', color='Compound',
        title='Stint Length by Tyre Compound'
    )
    fig.update_layout(template='plotly_dark', yaxis_title='Stint Length (laps)')
    fig.show()
else:
    print('No stint data available')

## 6. Telemetry Analysis

Deep dive into speed, throttle, and brake telemetry data.

In [ ]:
# Get telemetry for race winner's fastest lap
def get_driver_telemetry(session, driver_code: str):
    """Get telemetry data for a driver's fastest lap."""
    if session is None:
        return None, None
    try:
        driver_laps = session.laps.pick_driver(driver_code)
        fastest_lap = driver_laps.pick_fastest()
        telemetry = fastest_lap.get_telemetry()
        return fastest_lap, telemetry
    except Exception as e:
        print(f'Error getting telemetry for {driver_code}: {e}')
        return None, None

if session is not None and not results.empty:
    winner = results.iloc[0]['Abbreviation']
    winner_lap, telemetry = get_driver_telemetry(session, winner)
    
    if telemetry is not None and not telemetry.empty:
        print(f'Telemetry data for {winner} fastest lap:')
        print(f'Data points: {len(telemetry)}')
        display(telemetry.head())
    else:
        print('Could not load telemetry data')
else:
    print('Session or results not available')

In [ ]:
# Speed trace visualization
if 'telemetry' in dir() and telemetry is not None and not telemetry.empty:
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=telemetry['Distance'], y=telemetry['Speed'],
        mode='lines', name='Speed',
        line=dict(color='cyan', width=1)
    ))
    fig.update_layout(
        title=f'{winner} Speed Trace - Fastest Lap',
        xaxis_title='Distance (m)',
        yaxis_title='Speed (km/h)',
        template='plotly_dark'
    )
    fig.show()
else:
    print('No telemetry data for speed trace')

In [ ]:
# Combined telemetry plot (Speed, Throttle, Brake)
if 'telemetry' in dir() and telemetry is not None and not telemetry.empty:
    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        subplot_titles=('Speed', 'Throttle', 'Brake'),
        vertical_spacing=0.08
    )
    
    # Speed trace
    fig.add_trace(go.Scatter(
        x=telemetry['Distance'], y=telemetry['Speed'],
        line=dict(color='cyan'), name='Speed'
    ), row=1, col=1)
    
    # Throttle trace
    fig.add_trace(go.Scatter(
        x=telemetry['Distance'], y=telemetry['Throttle'],
        line=dict(color='green'), name='Throttle'
    ), row=2, col=1)
    
    # Brake trace (convert to percentage)
    brake_pct = telemetry['Brake'].astype(float) * 100
    fig.add_trace(go.Scatter(
        x=telemetry['Distance'], y=brake_pct,
        line=dict(color='red'), name='Brake'
    ), row=3, col=1)
    
    fig.update_layout(
        height=700,
        title_text=f'{winner} Telemetry - Fastest Lap',
        template='plotly_dark',
        showlegend=False
    )
    fig.update_xaxes(title_text='Distance (m)', row=3, col=1)
    fig.show()
else:
    print('No telemetry data for combined plot')

## 7. Driver Comparison

Head-to-head comparison between two drivers.

In [ ]:
# Compare P1 vs P2 lap times
if session is not None and not results.empty and len(results) >= 2:
    driver1 = results.iloc[0]['Abbreviation']
    driver2 = results.iloc[1]['Abbreviation']
    
    try:
        # Get lap times for both drivers
        d1_laps = session.laps.pick_driver(driver1)[['LapNumber', 'LapTime']].copy()
        d2_laps = session.laps.pick_driver(driver2)[['LapNumber', 'LapTime']].copy()
        
        d1_laps['LapTime_sec'] = d1_laps['LapTime'].dt.total_seconds()
        d2_laps['LapTime_sec'] = d2_laps['LapTime'].dt.total_seconds()
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=d1_laps['LapNumber'], y=d1_laps['LapTime_sec'],
            mode='lines+markers', name=driver1
        ))
        fig.add_trace(go.Scatter(
            x=d2_laps['LapNumber'], y=d2_laps['LapTime_sec'],
            mode='lines+markers', name=driver2
        ))
        
        fig.update_layout(
            title=f'{driver1} vs {driver2} - Lap Times Comparison',
            xaxis_title='Lap',
            yaxis_title='Lap Time (sec)',
            template='plotly_dark'
        )
        fig.show()
    except Exception as e:
        print(f'Error comparing drivers: {e}')
else:
    print('Not enough data for driver comparison')

In [ ]:
# Speed comparison on fastest laps
if session is not None and not results.empty and len(results) >= 2:
    driver1 = results.iloc[0]['Abbreviation']
    driver2 = results.iloc[1]['Abbreviation']
    
    try:
        _, d1_tel = get_driver_telemetry(session, driver1)
        _, d2_tel = get_driver_telemetry(session, driver2)
        
        if d1_tel is not None and d2_tel is not None:
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=d1_tel['Distance'], y=d1_tel['Speed'],
                mode='lines', name=driver1
            ))
            fig.add_trace(go.Scatter(
                x=d2_tel['Distance'], y=d2_tel['Speed'],
                mode='lines', name=driver2
            ))
            
            fig.update_layout(
                title=f'Speed Comparison: {driver1} vs {driver2}',
                xaxis_title='Distance (m)',
                yaxis_title='Speed (km/h)',
                template='plotly_dark'
            )
            fig.show()
        else:
            print('Could not load telemetry for comparison')
    except Exception as e:
        print(f'Error in speed comparison: {e}')
else:
    print('Not enough data for speed comparison')

## 8. Weather Analysis

Analyze weather conditions during the race.

In [ ]:
# Get weather data
def get_weather_data(session) -> pd.DataFrame:
    """Extract weather data from session."""
    if session is None:
        return pd.DataFrame()
    try:
        weather = session.weather_data.copy()
        return weather
    except Exception as e:
        print(f'Error getting weather data: {e}')
        return pd.DataFrame()

weather = get_weather_data(session)
if not weather.empty:
    print('Weather Data Sample:')
    display(weather.head(10))
else:
    print('No weather data available')

In [ ]:
# Temperature evolution during race
if not weather.empty and 'AirTemp' in weather.columns:
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        subplot_titles=('Air Temperature', 'Track Temperature'),
        vertical_spacing=0.1
    )
    
    fig.add_trace(go.Scatter(
        y=weather['AirTemp'], mode='lines',
        line=dict(color='skyblue'), name='Air Temp'
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        y=weather['TrackTemp'], mode='lines',
        line=dict(color='orange'), name='Track Temp'
    ), row=2, col=1)
    
    fig.update_layout(
        height=500,
        title_text='Temperature During Race',
        template='plotly_dark',
        showlegend=False
    )
    fig.update_yaxes(title_text='Air Temp (C)', row=1, col=1)
    fig.update_yaxes(title_text='Track Temp (C)', row=2, col=1)
    fig.show()
else:
    print('No temperature data available')

## 9. Sector Times Analysis

Compare best sector times between drivers.

In [ ]:
# Best sector times for top 10
if session is not None and not results.empty and not laps.empty:
    top_10 = results.head(10)['Abbreviation'].tolist()
    top_10_laps = laps[laps['Driver'].isin(top_10)].copy()
    
    sector_cols = ['Sector1Time', 'Sector2Time', 'Sector3Time']
    
    # Check if sector columns exist and convert to seconds
    if all(col in top_10_laps.columns for col in sector_cols):
        for col in sector_cols:
            top_10_laps[f'{col}_sec'] = top_10_laps[col].dt.total_seconds()
        
        # Best sector times per driver
        best_sectors = top_10_laps.groupby('Driver').agg({
            'Sector1Time_sec': 'min',
            'Sector2Time_sec': 'min',
            'Sector3Time_sec': 'min'
        }).round(3)
        
        print('Best Sector Times (Top 10 Finishers):')
        display(best_sectors)
    else:
        print('Sector time columns not available')
else:
    print('No data for sector analysis')

In [ ]:
# Sector times heatmap
if 'best_sectors' in dir() and not best_sectors.empty:
    fig = px.imshow(
        best_sectors.T,
        title='Best Sector Times Heatmap',
        labels={'color': 'Time (sec)'},
        color_continuous_scale='RdYlGn_r',
        aspect='auto'
    )
    fig.update_layout(template='plotly_dark')
    fig.show()
else:
    print('No sector data for heatmap')

## 10. Position Changes Throughout Race

Animated visualization of position changes lap by lap.

In [ ]:
# Position changes animation
if not laps.empty and 'Position' in laps.columns:
    position_data = laps[['Driver', 'LapNumber', 'Position']].dropna()
    
    if not position_data.empty:
        # Ensure Position is numeric
        position_data['Position'] = pd.to_numeric(position_data['Position'], errors='coerce')
        position_data = position_data.dropna()
        
        fig = px.bar(
            position_data, x='Position', y='Driver',
            animation_frame='LapNumber',
            orientation='h',
            title='Position Changes Throughout Race',
            range_x=[0, 21]
        )
        fig.update_layout(
            template='plotly_dark',
            yaxis={'categoryorder':'total ascending'},
            height=600
        )
        fig.show()
    else:
        print('No valid position data')
else:
    print('No position data available')

---

## Summary

This notebook demonstrates FastF1's capabilities for F1 telemetry analysis:

| Feature | Description |
|---------|-------------|
| Session Loading | Load any F1 session (Race, Qualifying, Practice) |
| Lap Times | Average, fastest, and evolution analysis |
| Telemetry | Speed, throttle, brake traces |
| Tyre Strategy | Compound usage and stint analysis |
| Weather | Temperature and conditions |
| Sector Times | S1, S2, S3 comparison |
| Driver Comparison | Head-to-head analysis |

**Note:** Data availability depends on race completion and FastF1 API status.

---
*Created with FastF1 Telemetry Analysis Pipeline*

## 1. Get 2025 Season Schedule

FastF1 menyediakan akses ke data telemetri F1 resmi. Di sini kita load kalender 2025.

In [ ]:
# Get 2025 schedule
schedule = get_schedule(2025)
print('2025 F1 Calendar:')
schedule[['RoundNumber', 'EventName', 'Country', 'Location', 'EventDate']]

2025 F1 Calendar:


,RoundNumber,EventName,Country,Location,EventDate
0,0,Pre-Season Testing,Bahrain,Sakhir,2025-02-28
1,1,Australian Grand Prix,Australia,Melbourne,2025-03-16
2,2,Chinese Grand Prix,China,Shanghai,2025-03-23
3,3,Japanese Grand Prix,Japan,Suzuka,2025-04-06
4,4,Bahrain Grand Prix,Bahrain,Sakhir,2025-04-13
5,5,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,2025-04-20
6,6,Miami Grand Prix,United States,Miami Gardens,2025-05-04
7,7,Emilia Romagna Grand Prix,Italy,Imola,2025-05-18
8,8,Monaco Grand Prix,Monaco,Monaco,2025-05-25
9,9,Spanish Grand Prix,Spain,Barcelona,2025-06-01


## 2. Load Race Session Data

In [ ]:
# Load a race session (example: Australia 2025 - first race)
session = get_session(2025, 'Australia', 'R')
print(f'Session loaded: {session.event["EventName"]} - {session.name}')

core           INFO 	Loading data for Australian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core    

Session loaded: Australian Grand Prix - Race


In [ ]:
# Get race results
results = get_race_results(session)
print('Race Results:')
results[['Position', 'Abbreviation', 'TeamName', 'Time', 'Status', 'Points']]

Race Results:


,Position,Abbreviation,TeamName,Time,Status,Points
4,1.0,NOR,McLaren,0 days 01:42:06.304000,Finished,25.0
1,2.0,VER,Red Bull Racing,0 days 00:00:00.895000,Finished,18.0
63,3.0,RUS,Mercedes,0 days 00:00:08.481000,Finished,15.0
12,4.0,ANT,Mercedes,0 days 00:00:10.135000,Finished,12.0
23,5.0,ALB,Williams,0 days 00:00:12.773000,Finished,10.0
18,6.0,STR,Aston Martin,0 days 00:00:17.413000,Finished,8.0
27,7.0,HUL,Kick Sauber,0 days 00:00:18.423000,Finished,6.0
16,8.0,LEC,Ferrari,0 days 00:00:19.826000,Finished,4.0
81,9.0,PIA,McLaren,0 days 00:00:20.448000,Finished,2.0
44,10.0,HAM,Ferrari,0 days 00:00:22.473000,Finished,1.0


## 3. Lap Time Analysis

In [ ]:
# Get lap data
laps = get_lap_data(session)
print(f'Total laps: {len(laps)}')
laps.head()

Total laps: 927


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:13:00.002000,VER,1,0 days 00:01:59.392000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:20.705000,...,True,Red Bull Racing,0 days 01:11:00.355000,2025-03-16 04:18:22.974,124,2.0,False,,False,False
1,0 days 01:15:49.358000,VER,1,NaT,2.0,1.0,NaT,0 days 01:15:38.205000,0 days 00:00:58.141000,0 days 00:00:37.976000,...,True,Red Bull Racing,0 days 01:13:00.002000,2025-03-16 04:20:22.621,4,2.0,False,,False,False
2,0 days 01:18:31.526000,VER,1,NaT,3.0,2.0,0 days 01:15:51.658000,0 days 01:18:20.223000,0 days 00:00:56.230000,0 days 00:00:33.683000,...,False,Red Bull Racing,0 days 01:15:49.358000,2025-03-16 04:23:11.977,4,2.0,False,,False,False
3,0 days 01:21:07.226000,VER,1,NaT,4.0,3.0,0 days 01:18:34.029000,0 days 01:20:56.543000,0 days 00:00:54.351000,0 days 00:00:32.712000,...,False,Red Bull Racing,0 days 01:18:31.526000,2025-03-16 04:25:54.145,4,2.0,False,,False,False
4,0 days 01:23:30.835000,VER,1,0 days 00:02:23.609000,5.0,4.0,0 days 01:21:09.534000,NaT,0 days 00:00:53.513000,0 days 00:00:32.627000,...,False,Red Bull Racing,0 days 01:21:07.226000,2025-03-16 04:28:29.845,4,2.0,False,,False,False


In [ ]:
# Convert lap times to seconds for easier analysis
laps['LapTime_sec'] = laps['LapTime'].dt.total_seconds()

# Filter valid laps (remove outliers like pit laps, safety car, etc.)
valid_laps = laps[(laps['LapTime_sec'] > 80) & (laps['LapTime_sec'] < 120)]

# Average lap time per driver
avg_lap_times = valid_laps.groupby('Driver')['LapTime_sec'].mean().sort_values()

fig = px.bar(avg_lap_times.reset_index(name='Avg Lap Time (sec)'),
             x='Avg Lap Time (sec)', y='Driver', orientation='h',
             color='Avg Lap Time (sec)', color_continuous_scale='RdYlGn_r',
             title='Average Lap Time per Driver')
fig.update_layout(yaxis={'categoryorder':'total ascending'}, template='plotly_dark')
fig.show()

In [ ]:
# Lap time evolution
top_3_drivers = results.head(3)['Abbreviation'].tolist()
top_3_laps = valid_laps[valid_laps['Driver'].isin(top_3_drivers)]

fig = px.line(top_3_laps, x='LapNumber', y='LapTime_sec', color='Driver',
              title='Lap Time Evolution - Top 3 Finishers')
fig.update_layout(template='plotly_dark', xaxis_title='Lap', yaxis_title='Lap Time (sec)')
fig.show()

c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## 4. Fastest Laps Comparison

In [ ]:
# Get fastest lap for each driver
fastest_laps = laps.loc[laps.groupby('Driver')['LapTime_sec'].idxmin()]
fastest_laps = fastest_laps.sort_values('LapTime_sec')[['Driver', 'Team', 'LapNumber', 'LapTime_sec', 'Compound']]

print('Fastest Lap per Driver:')
fastest_laps

KeyError: '[nan] not in index'

In [ ]:
# Gap to fastest lap
fastest_overall = fastest_laps['LapTime_sec'].min()
fastest_laps['Gap_to_Fastest'] = fastest_laps['LapTime_sec'] - fastest_overall

fig = px.bar(fastest_laps, x='Gap_to_Fastest', y='Driver', orientation='h',
             color='Team', title='Gap to Fastest Lap')
fig.update_layout(yaxis={'categoryorder':'total ascending'}, template='plotly_dark')
fig.show()

## 5. Tyre Strategy Analysis

In [ ]:
# Compound usage per driver
compound_usage = laps.groupby(['Driver', 'Compound']).size().unstack(fill_value=0)

fig = px.imshow(compound_usage, 
                title='Tyre Compound Usage (Laps per Compound)',
                labels={'color': 'Laps'},
                color_continuous_scale='YlOrRd')
fig.update_layout(template='plotly_dark')
fig.show()

In [ ]:
# Stint length by compound
stint_data = laps.groupby(['Driver', 'Stint'])['Compound'].first().reset_index()
stint_lengths = laps.groupby(['Driver', 'Stint']).size().reset_index(name='StintLength')
stint_data = stint_data.merge(stint_lengths, on=['Driver', 'Stint'])

fig = px.box(stint_data, x='Compound', y='StintLength', color='Compound',
             title='Stint Length by Tyre Compound')
fig.update_layout(template='plotly_dark')
fig.show()

## 6. Telemetry Analysis (Speed, Throttle, Brake)

In [ ]:
# Get telemetry for fastest lap
winner = results.iloc[0]['Abbreviation']
winner_fastest = laps.pick_driver(winner).pick_fastest()

telemetry = get_telemetry(winner_fastest)
print(f'Telemetry data for {winner} fastest lap:')
print(f'Data points: {len(telemetry)}')
telemetry.head()

In [ ]:
# Speed trace
fig = go.Figure()
fig.add_trace(go.Scatter(x=telemetry['Distance'], y=telemetry['Speed'],
                         mode='lines', name='Speed'))
fig.update_layout(title=f'{winner} Speed Trace - Fastest Lap',
                  xaxis_title='Distance (m)', yaxis_title='Speed (km/h)',
                  template='plotly_dark')
fig.show()

In [ ]:
# Combined telemetry plot
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    subplot_titles=('Speed', 'Throttle', 'Brake'),
                    vertical_spacing=0.08)

fig.add_trace(go.Scatter(x=telemetry['Distance'], y=telemetry['Speed'],
                         line=dict(color='cyan'), name='Speed'), row=1, col=1)
fig.add_trace(go.Scatter(x=telemetry['Distance'], y=telemetry['Throttle'],
                         line=dict(color='green'), name='Throttle'), row=2, col=1)
fig.add_trace(go.Scatter(x=telemetry['Distance'], y=telemetry['Brake'].astype(int) * 100,
                         line=dict(color='red'), name='Brake'), row=3, col=1)

fig.update_layout(height=700, title_text=f'{winner} Telemetry - Fastest Lap',
                  template='plotly_dark', showlegend=False)
fig.update_xaxes(title_text='Distance (m)', row=3, col=1)
fig.show()

## 7. Driver Comparison

In [ ]:
# Compare two drivers lap times
driver1 = results.iloc[0]['Abbreviation']
driver2 = results.iloc[1]['Abbreviation']

# Get lap times for both drivers
d1_laps = laps.pick_driver(driver1)[['LapNumber', 'LapTime']].copy()
d2_laps = laps.pick_driver(driver2)[['LapNumber', 'LapTime']].copy()

d1_laps['LapTime_sec'] = d1_laps['LapTime'].dt.total_seconds()
d2_laps['LapTime_sec'] = d2_laps['LapTime'].dt.total_seconds()

fig = go.Figure()
fig.add_trace(go.Scatter(x=d1_laps['LapNumber'], y=d1_laps['LapTime_sec'],
                         mode='lines+markers', name=driver1))
fig.add_trace(go.Scatter(x=d2_laps['LapNumber'], y=d2_laps['LapTime_sec'],
                         mode='lines+markers', name=driver2))
fig.update_layout(title=f'{driver1} vs {driver2} Lap Times - Australia 2025',
                  xaxis_title='Lap', yaxis_title='Lap Time (sec)',
                  template='plotly_dark')
fig.show()

In [ ]:
# Speed comparison on fastest laps
d1_fastest = laps.pick_driver(driver1).pick_fastest()
d2_fastest = laps.pick_driver(driver2).pick_fastest()

d1_tel = get_telemetry(d1_fastest)
d2_tel = get_telemetry(d2_fastest)

fig = go.Figure()
fig.add_trace(go.Scatter(x=d1_tel['Distance'], y=d1_tel['Speed'],
                         mode='lines', name=driver1))
fig.add_trace(go.Scatter(x=d2_tel['Distance'], y=d2_tel['Speed'],
                         mode='lines', name=driver2))
fig.update_layout(title=f'Speed Comparison: {driver1} vs {driver2} - Australia 2025',
                  xaxis_title='Distance (m)', yaxis_title='Speed (km/h)',
                  template='plotly_dark')
fig.show()

## 8. Weather Analysis

In [ ]:
# Get weather data
weather = get_weather_data(session)
print('Weather Data:')
weather.head(10)

In [ ]:
# Temperature evolution during race
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=('Air Temperature', 'Track Temperature'),
                    vertical_spacing=0.1)

fig.add_trace(go.Scatter(y=weather['AirTemp'], mode='lines',
                         line=dict(color='skyblue'), name='Air Temp'), row=1, col=1)
fig.add_trace(go.Scatter(y=weather['TrackTemp'], mode='lines',
                         line=dict(color='orange'), name='Track Temp'), row=2, col=1)

fig.update_layout(height=500, title_text='Temperature During Race',
                  template='plotly_dark', showlegend=False)
fig.update_yaxes(title_text='Air Temp (C)', row=1, col=1)
fig.update_yaxes(title_text='Track Temp (C)', row=2, col=1)
fig.show()

## 9. Sector Times Analysis

In [ ]:
# Sector times for top 10
top_10 = results.head(10)['Abbreviation'].tolist()
top_10_laps = laps[laps['Driver'].isin(top_10)]

# Convert sector times
for s in ['Sector1Time', 'Sector2Time', 'Sector3Time']:
    top_10_laps[f'{s}_sec'] = top_10_laps[s].dt.total_seconds()

# Best sector times per driver
best_sectors = top_10_laps.groupby('Driver').agg({
    'Sector1Time_sec': 'min',
    'Sector2Time_sec': 'min',
    'Sector3Time_sec': 'min'
}).round(3)

print('Best Sector Times (Top 10 Finishers):')
best_sectors

In [ ]:
# Sector time heatmap
fig = px.imshow(best_sectors.T, 
                title='Best Sector Times Heatmap',
                labels={'color': 'Time (sec)'},
                color_continuous_scale='RdYlGn_r',
                aspect='auto')
fig.update_layout(template='plotly_dark')
fig.show()

## 10. Position Changes Animation

In [ ]:
# Position per lap for all drivers
position_data = laps[['Driver', 'LapNumber', 'Position']].dropna()

# Create animated bar chart race
fig = px.bar(position_data, x='Position', y='Driver', 
             animation_frame='LapNumber',
             orientation='h',
             title='Position Changes Throughout Race',
             range_x=[0, 21])
fig.update_layout(template='plotly_dark', 
                  yaxis={'categoryorder':'total ascending'},
                  height=600)
fig.show()

---
## Summary

Notebook ini menggunakan FastF1 untuk analisis lanjutan:
- Session dan race results loading
- Lap time analysis dan comparison
- Telemetry data (speed, throttle, brake)
- Tyre strategy analysis
- Weather data
- Sector times comparison
- Driver head-to-head comparison

FastF1 menyediakan data resmi F1 yang lebih detail dibanding CSV dataset.